In [1]:
import sys

sys.path.insert(0, "../")

import pandas as pd
from mdu.eval.table_analysis_utils import (
    transform_by_tasks,
    select_composite_and_components,
    check_composite_dominance,
    compute_average_ranks,
    analyze_composite_pareto_performance,
)

# Set pandas display options to show all columns
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/nikita/Programming/multidimensional_uncertainty/.venv/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/nikita/Programming/multidimensional_uncertainty/.venv/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/nikita/Programming/multidimensional_uncertainty/.venv/lib

In [2]:
df = pd.read_csv(
    "../resources/extended_benchmark/extra_extended_benchmark_entropic_target_beta_eps_0.5_iters_150_tol_1e-06_rs_42_grid_size_5_n_targets_multiplier_1.csv"
)
# "../resources/extended_benchmark/extended_benchmark_entropic_target_exp_eps_0.5_iters_150_tol_1e-06_rs_42_grid_size_5_n_targets_multiplier_1_global_scaler.csv"
# "../resources/extended_benchmark/extended_benchmark_entropic_target_beta_eps_0.5_iters_150_tol_1e-06_rs_42_grid_size_5_n_targets_multiplier_1.csv"

In [3]:
df.head(10)

,ind_dataset,ood_dataset,measure,uncertainty_type,gname,risk_type,gt_approximation,pred_approximation,ensemble_group,problem_type,roc_auc,average_precision,accuracy,aurc,acc_cov_auc,coverage_at_1pct_error,coverage_at_2pct_error,coverage_at_5pct_error,n_ind_samples,n_ood_samples,n_correct,n_incorrect,ensemble_accuracy
0,cifar10,cifar100,Risk_LogScore_TotalRisk_outer_outer,Risk,LogScore,TotalRisk,outer,outer,0,ood_detection,0.912015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7200,10000.0,NaN,NaN,0.963750
1,cifar10,cifar10,Risk_LogScore_TotalRisk_outer_outer,Risk,LogScore,TotalRisk,outer,outer,0,misclassification_detection,0.944999,0.381728,0.963750,NaN,NaN,NaN,NaN,NaN,7200,NaN,6939.0,261.0,0.963750
2,cifar10,cifar10,Risk_LogScore_TotalRisk_outer_outer,Risk,LogScore,TotalRisk,outer,outer,0,selective_prediction,NaN,NaN,0.963750,0.002928,0.996933,0.000139,0.000139,0.000139,7200,NaN,NaN,NaN,0.963750
3,cifar10,cifar100,Risk_LogScore_TotalRisk_outer_outer,Risk,LogScore,TotalRisk,outer,outer,1,ood_detection,0.910690,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7200,10000.0,NaN,NaN,0.964306
4,cifar10,cifar10,Risk_LogScore_TotalRisk_outer_outer,Risk,LogScore,TotalRisk,outer,outer,1,misclassification_detection,0.944610,0.374033,0.964306,NaN,NaN,NaN,NaN,NaN,7200,NaN,6943.0,257.0,0.964306
5,cifar10,cifar10,Risk_LogScore_TotalRisk_outer_outer,Risk,LogScore,TotalRisk,outer,outer,1,selective_prediction,NaN,NaN,0.964306,0.002930,0.996931,0.000139,0.000139,0.000139,7200,NaN,NaN,NaN,0.964306
6,cifar10,cifar100,Risk_LogScore_TotalRisk_outer_outer,Risk,LogScore,TotalRisk,outer,outer,2,ood_detection,0.911888,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7200,10000.0,NaN,NaN,0.963333
7,cifar10,cifar10,Risk_LogScore_TotalRisk_outer_outer,Risk,LogScore,TotalRisk,outer,outer,2,misclassification_detection,0.943641,0.360288,0.963333,NaN,NaN,NaN,NaN,NaN,7200,NaN,6936.0,264.0,0.963333
8,cifar10,cifar10,Risk_LogScore_TotalRisk_outer_outer,Risk,LogScore,TotalRisk,outer,outer,2,selective_prediction,NaN,NaN,0.963333,0.003023,0.996838,0.000139,0.000139,0.000139,7200,NaN,NaN,NaN,0.963333
9,cifar10,cifar100,Risk_LogScore_TotalRisk_outer_outer,Risk,LogScore,TotalRisk,outer,outer,3,ood_detection,0.911436,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7200,10000.0,NaN,NaN,0.964167


In [4]:
df.ood_dataset.unique()

array(['cifar100', 'cifar10', 'tiny_imagenet', 'svhn', 'imagenet_a',
       'imagenet_o', 'imagenet_r'], dtype=object)

In [5]:
transformed_df = transform_by_tasks(df)

In [6]:
transformed_df

measure                                  R_b 1 (Brier)  R_b 1 (Logscore)  \
ind_dataset   eval                                                         
cifar10       cifar10 [miscls]                0.942466          0.942267   
              cifar10 [selective]             0.996836          0.996818   
              cifar100 [ood]                  0.914436          0.916906   
              svhn [ood]                      0.958357          0.962992   
              tiny_imagenet [ood]             0.908611          0.911364   
cifar100      cifar10 [ood]                   0.773000          0.773270   
              cifar100 [miscls]               0.857773          0.845094   
              cifar100 [selective]            0.920203          0.915943   
              svhn [ood]                      0.858293          0.870120   
              tiny_imagenet [ood]             0.789521          0.809888   
tiny_imagenet tiny_imagenet [miscls]          0.854657          0.844739   
              tiny_imagenet [selective]       0.892676          0.888853   
              imagenet_a [ood]                0.827188          0.835350   
              imagenet_r [ood]                0.816241          0.825339   
              imagenet_o [ood]                0.720970          0.724312   

measure                                  R_b 1 (Spherical)  R_b 1 (Zero-one)  \
ind_dataset   eval                                                             
cifar10       cifar10 [miscls]                    0.942228          0.941776   
              cifar10 [selective]                 0.996827          0.996810   
              cifar100 [ood]                      0.914506          0.913191   
              svhn [ood]                          0.958530          0.956294   
              tiny_imagenet [ood]                 0.908679          0.907210   
cifar100      cifar10 [ood]                       0.773388          0.772177   
              cifar100 [miscls]                   0.856181          0.858986   
              cifar100 [selective]                0.919719          0.920576   
              svhn [ood]                          0.860970          0.855937   
              tiny_imagenet [ood]                 0.806267          0.803022   
tiny_imagenet tiny_imagenet [miscls]              0.853150          0.854657   
              tiny_imagenet [selective]           0.892115          0.892648   
              imagenet_a [ood]                    0.830074          0.826338   
              imagenet_r [ood]                    0.819225          0.815484   
              imagenet_o [ood]                    0.722622          0.721199   

measure                                  R_b 2 (Brier)  R_b 2 (Logscore)  \
ind_dataset   eval                                                         
cifar10       cifar10 [miscls]                0.945782          0.944886   
              cifar10 [selective]             0.996959          0.996920   
              cifar100 [ood]                  0.913049          0.916237   
              svhn [ood]                      0.957686          0.963150   
              tiny_imagenet [ood]             0.905817          0.909549   
cifar100      cifar10 [ood]                   0.774385          0.774538   
              cifar100 [miscls]               0.865555          0.850089   
              cifar100 [selective]            0.922462          0.917541   
              svhn [ood]                      0.856738          0.870178   
              tiny_imagenet [ood]             0.937085          0.926019   
tiny_imagenet tiny_imagenet [miscls]          0.863376          0.849698   
              tiny_imagenet [selective]       0.895661          0.890666   
              imagenet_a [ood]                0.834463          0.840955   
              imagenet_r [ood]                0.824133          0.830950   
              imagenet_o [ood]                0.735803          0.734891   

measure                                  R_b 2 (Spher

In [7]:
transformed_df.shape

(15, 99)

In [8]:
# Count unique non-composite measures of uncertainty
non_composite_columns = [col for col in transformed_df.columns if not col.startswith('composite')]
print(f"Number of unique non-composite measures: {len(non_composite_columns)}")
print("\nNon-composite measures:")
for col in non_composite_columns:
    print(f"  - {col}")


Number of unique non-composite measures: 86

Non-composite measures:
  - R_b 1 (Brier)
  - R_b 1 (Logscore)
  - R_b 1 (Spherical)
  - R_b 1 (Zero-one)
  - R_b 2 (Brier)
  - R_b 2 (Logscore)
  - R_b 2 (Spherical)
  - R_b 2 (Zero-one)
  - R_b 3 (Brier)
  - R_b 3 (Logscore)
  - R_b 3 (Spherical)
  - R_b 3 (Zero-one)
  - R_e 1 1 (Brier)
  - R_e 1 1 (Logscore)
  - R_e 1 1 (Spherical)
  - R_e 1 1 (Zero-one)
  - R_e 1 2 (Brier)
  - R_e 1 2 (Logscore)
  - R_e 1 2 (Spherical)
  - R_e 1 2 (Zero-one)
  - R_e 1 3 (Brier)
  - R_e 1 3 (Logscore)
  - R_e 1 3 (Spherical)
  - R_e 1 3 (Zero-one)
  - R_e 2 1 (Brier)
  - R_e 2 1 (Logscore)
  - R_e 2 1 (Spherical)
  - R_e 2 1 (Zero-one)
  - R_e 2 2 (Brier)
  - R_e 2 2 (Logscore)
  - R_e 2 2 (Spherical)
  - R_e 2 2 (Zero-one)
  - R_e 2 3 (Brier)
  - R_e 2 3 (Logscore)
  - R_e 2 3 (Spherical)
  - R_e 2 3 (Zero-one)
  - R_e 3 1 (Brier)
  - R_e 3 1 (Logscore)
  - R_e 3 1 (Spherical)
  - R_e 3 1 (Zero-one)
  - R_e 3 2 (Brier)
  - R_e 3 2 (Logscore)
  - R_e 3 2 

In [9]:
from configs.interesting_compositions import INTERESTING_COMPOSITIONS

for k in INTERESTING_COMPOSITIONS.keys():
    print(k)

COMPOSITE BAYES ALL OUTER
COMPOSITE BAYES ALL INNER
COMPOSITE BAYES ALL CENTRAL
COMPOSITE BAYES (LBS) OUTER
COMPOSITE BAYES (LBS) INNER
COMPOSITE BAYES (LBS) CENTRAL
COMPOSITE BAYES ALL OUTER + M
COMPOSITE BAYES ALL INNER + M
COMPOSITE BAYES ALL CENTRAL + M
COMPOSITE BAYES (LBS) OUTER + M
COMPOSITE BAYES (LBS) INNER + M
COMPOSITE BAYES (LBS) CENTRAL + M
COMPOSITE BAYES ALL OUTER + GMM
COMPOSITE BAYES ALL INNER + GMM
COMPOSITE BAYES ALL CENTRAL + GMM
COMPOSITE BAYES (LBS) OUTER + GMM
COMPOSITE BAYES (LBS) INNER + GMM
COMPOSITE BAYES (LBS) CENTRAL + GMM
COMPOSITE EXCESS ALL OUTER OUTER
COMPOSITE EXCESS ALL OUTER INNER
COMPOSITE EXCESS ALL OUTER CENTRAL
COMPOSITE EXCESS ALL INNER OUTER
COMPOSITE EXCESS ALL INNER INNER
COMPOSITE EXCESS ALL INNER CENTRAL
COMPOSITE EXCESS ALL CENTRAL OUTER
COMPOSITE EXCESS ALL CENTRAL INNER
COMPOSITE EXCESS ALL CENTRAL CENTRAL
COMPOSITE EXCESS LBS OUTER OUTER
COMPOSITE EXCESS LBS OUTER INNER
COMPOSITE EXCESS LBS OUTER CENTRAL
COMPOSITE EXCESS LBS INNER OUTER

In [10]:
res_df = select_composite_and_components(transformed_df, "COMPOSITE EAT LOGSCORE OUTER OUTER + M")

res_df

Note: Composite measure 'composite eat logscore outer outer + m' not found in DataFrame columns


measure                                  R_e 1 1 (Logscore)  \
ind_dataset   eval                                            
cifar10       cifar10 [miscls]                     0.940430   
              cifar10 [selective]                  0.996843   
              cifar100 [ood]                       0.904730   
              svhn [ood]                           0.942616   
              tiny_imagenet [ood]                  0.895731   
cifar100      cifar10 [ood]                        0.724548   
              cifar100 [miscls]                    0.818006   
              cifar100 [selective]                 0.909667   
              svhn [ood]                           0.755848   
              tiny_imagenet [ood]                  0.999852   
tiny_imagenet tiny_imagenet [miscls]               0.813053   
              tiny_imagenet [selective]            0.879249   
              imagenet_a [ood]                     0.781130   
              imagenet_r [ood]                     0.774366   
              imagenet_o [ood]                     0.752832   

measure                                  R_t 1 1 (Logscore)  R_b 1 (Logscore)  \
ind_dataset   eval                                                              
cifar10       cifar10 [miscls]                     0.943244          0.942267   
              cifar10 [selective]                  0.996864          0.996818   
              cifar100 [ood]                       0.911507          0.916906   
              svhn [ood]                           0.956506          0.962992   
              tiny_imagenet [ood]                  0.903615          0.911364   
cifar100      cifar10 [ood]                        0.774023          0.773270   
              cifar100 [miscls]                    0.853064          0.845094   
              cifar100 [selective]                 0.918425          0.915943   
              svhn [ood]                           0.867727          0.870120   
              tiny_imagenet [ood]                  0.999997          0.809888   
tiny_imagenet tiny_imagenet [miscls]               0.850815          0.844739   
              tiny_imagenet [selective]            0.890910          0.888853   
              imagenet_a [ood]                     0.846305          0.835350   
              imagenet_r [ood]                     0.836572          0.825339   
              imagenet_o [ood]                     0.753524          0.724312   

measure                                  mahalanobis  
ind_dataset   eval                                    
cifar10       cifar10 [miscls]              0.927621  
              cifar10 [selective]           0.996266  
              cifar100 [ood]                0.912238  
              svhn [ood]                    0.934311  
              tiny_imagenet [ood]           0.910273  
cifar100      cifar10 [ood]                 0.534822  
              cifar100 [miscls]             0.573908  
              cifar100 [selective]          0.810550  
              svhn [ood]                    0.678832  
              tiny_imagenet [ood]           0.622941  
tiny_imagenet tiny_imagenet [miscls]        0.416754  
              tiny_imagenet [selective]     0.659777  
              imagenet_a [ood]              0.440974  
              imagenet_r [ood]              0.404755  
              imagenet_o [ood]              0.512686

In [11]:
# Apply the dominance check
res_df_with_dominance = check_composite_dominance(res_df)
res_df_with_dominance

ValueError: No column starting with 'composite' found in DataFrame

In [ ]:
# Compute and display average ranks
avg_ranks = compute_average_ranks(transformed_df)

In [ ]:
for name, el in avg_ranks.items():
    print(name, round(el, 3))

composite bayes all central + m 20.467
R_t 2 3 (Spherical) 20.533
R_t 1 3 (Spherical) 20.6
composite bayes (lbs) central + m 20.6
composite eat logscore outer outer + m 21.467
composite bayes (lbs) inner 21.933
composite bayes all inner 22.0
R_t 2 1 (Spherical) 22.0
R_t 2 2 (Spherical) 22.267
R_t 3 1 (Logscore) 22.267
R_b 2 (Spherical) 22.267
R_b 2 (Brier) 22.333
R_b 3 (Brier) 22.333
R_t 3 2 (Brier) 22.333
R_t 2 2 (Brier) 22.333
R_t 3 3 (Brier) 22.333
R_t 2 3 (Brier) 22.333
composite bayes (lbs) central 22.4
R_t 3 2 (Logscore) 22.533
R_t 1 1 (Spherical) 22.6
R_t 1 2 (Brier) 22.667
R_t 1 3 (Brier) 22.667
composite bayes all inner + m 22.867
R_t 1 2 (Spherical) 23.267
composite bayes (lbs) inner + m 23.933
composite bayes all central 24.333
R_t 2 1 (Logscore) 24.667
R_b 3 (Logscore) 24.8
R_t 3 3 (Logscore) 24.8
R_t 1 2 (Logscore) 25.067
R_t 1 1 (Logscore) 25.133
R_b 2 (Logscore) 25.2
R_t 2 2 (Logscore) 25.2
R_t 3 3 (Zero-one) 25.8
R_t 3 2 (Zero-one) 25.8
R_t 2 3 (Zero-one) 25.8
R_b 2 (Ze

In [ ]:
composite_pareto_results = analyze_composite_pareto_performance(
    transformed_df, INTERESTING_COMPOSITIONS
)

In [ ]:
composite_pareto_results

{'COMPOSITE BAYES ALL OUTER': {'pareto_count': 30,
  'total_problems': 105,
  'pareto_percentage': 28.57142857142857},
 'COMPOSITE BAYES ALL INNER': {'pareto_count': 57,
  'total_problems': 105,
  'pareto_percentage': 54.285714285714285},
 'COMPOSITE BAYES ALL CENTRAL': {'pareto_count': 44,
  'total_problems': 105,
  'pareto_percentage': 41.904761904761905},
 'COMPOSITE BAYES (LBS) OUTER': {'pareto_count': 57,
  'total_problems': 105,
  'pareto_percentage': 54.285714285714285},
 'COMPOSITE BAYES (LBS) INNER': {'pareto_count': 69,
  'total_problems': 105,
  'pareto_percentage': 65.71428571428571},
 'COMPOSITE BAYES (LBS) CENTRAL': {'pareto_count': 58,
  'total_problems': 105,
  'pareto_percentage': 55.23809523809524},
 'COMPOSITE BAYES ALL OUTER + M': {'pareto_count': 62,
  'total_problems': 105,
  'pareto_percentage': 59.04761904761905},
 'COMPOSITE BAYES ALL INNER + M': {'pareto_count': 69,
  'total_problems': 105,
  'pareto_percentage': 65.71428571428571},
 'COMPOSITE BAYES ALL CENTR

In [ ]:
# Display results summary
if composite_pareto_results:
    print("\n" + "=" * 80)
    print("COMPOSITE PARETO FRONT ANALYSIS SUMMARY")
    print("=" * 80)

    # Sort by pareto percentage
    sorted_results = sorted(
        composite_pareto_results.items(),
        key=lambda x: x[1]["pareto_percentage"],
        reverse=True,
    )

    for composite_name, results in sorted_results:
        print(
            f"{composite_name:40} {results['pareto_count']:3d}/{results['total_problems']:3d} ({results['pareto_percentage']:5.1f}%)"
        )


COMPOSITE PARETO FRONT ANALYSIS SUMMARY
COMPOSITE EXCESS ALL INNER INNER         105/105 (100.0%)
COMPOSITE EXCESS ALL CENTRAL CENTRAL     105/105 (100.0%)
COMPOSITE EXCESS LBS INNER INNER         105/105 (100.0%)
COMPOSITE EXCESS LBS CENTRAL CENTRAL     105/105 (100.0%)
COMPOSITE EXCESS ALL INNER INNER + M      99/105 ( 94.3%)
COMPOSITE EXCESS ALL CENTRAL CENTRAL + M  99/105 ( 94.3%)
COMPOSITE EAT LOGSCORE OUTER OUTER + M    87/105 ( 82.9%)
COMPOSITE EXCESS ALL INNER CENTRAL        77/105 ( 73.3%)
COMPOSITE EXCESS ALL CENTRAL INNER        77/105 ( 73.3%)
COMPOSITE EXCESS LBS INNER CENTRAL        77/105 ( 73.3%)
COMPOSITE EXCESS LBS CENTRAL INNER        77/105 ( 73.3%)
COMPOSITE BAYES (LBS) INNER               69/105 ( 65.7%)
COMPOSITE BAYES ALL INNER + M             69/105 ( 65.7%)
COMPOSITE EXCESS ALL CENTRAL INNER + M    68/105 ( 64.8%)
COMPOSITE EXCESS ALL INNER CENTRAL + M    66/105 ( 62.9%)
COMPOSITE EXCESS LBS INNER OUTER          65/105 ( 61.9%)
COMPOSITE BAYES (LBS) CENTRAL +